In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d50bc05082f39e17b60adcfa56dce24003b1e54568321ce194e1c2b12a4e830b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Data Exploration")\
        .getOrCreate()

spark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [ ]:
df = spark.read.format("csv").load("/content/drive/MyDrive/LSPDP Project/train_data.csv", header=True, inferSchema=True)

In [ ]:
df.count()

5531451

In [ ]:
df.select("customer_ID").distinct().count()

458913

#Sampling 10% of Dataset

In [ ]:
# Set the fraction you want to sample
fraction_to_sample = 0.1  # 10% in this example

# Randomly sample the DataFrame
sampled_df = df.sample(withReplacement=False, fraction=fraction_to_sample, seed=42)

# Define the path for the sampled CSV file
output_path = "/content/drive/MyDrive/LSPDP Project/sampled_train_data_features.csv"

# Export the sampled DataFrame to a CSV file
sampled_df.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save(output_path)


In [ ]:
files_dir = "/content/drive/MyDrive/LSPDP Project/sampled_train_data_features.csv"
csv_files = [file for file in os.listdir(files_dir) if file.endswith('.csv')]

# Read each CSV file into a pandas DataFrame
dfs = [pd.read_csv(os.path.join(files_dir, file)) for file in csv_files]

# Concatenate the DataFrames along rows
combined_df = pd.concat(dfs, ignore_index=True)

# Write the combined DataFrame to a new CSV file
combined_df.to_csv("combined_file.csv", index=False)

In [ ]:
combined_df.shape

(553840, 190)

In [ ]:
# Write the combined DataFrame to a new CSV file
combined_df.to_csv("/content/drive/MyDrive/LSPDP Project/sampled_train_data_features_combined.csv", index=False)

#Get Labels of the sampled Dataset

In [ ]:
train_labels =pd.read_csv("/content/drive/MyDrive/LSPDP Project/train_labels.csv")
sampled_df = pd.read_csv("/content/drive/MyDrive/LSPDP Project/sampled_train_data_features_combined.csv")

In [ ]:
train_labels.shape

(458913, 2)

In [ ]:
sampled_df['customer_ID'].nunique()

323851

In [ ]:
sampled_df_customer_id = sampled_df['customer_ID']

In [ ]:
sampled_customer_ids = set(sampled_df['customer_ID'])

In [ ]:
filtered_trained_labels = train_labels[train_labels['customer_ID'].isin(sampled_customer_ids)]

In [ ]:
filtered_trained_labels.shape

(323851, 2)

In [ ]:
filtered_trained_labels.to_csv("/content/drive/MyDrive/LSPDP Project/filtered_train_labels.csv", index=False)

#Preprocessing Training DF
## 1. Keeping imp features only
## 2. Customer_ID from hex to int
## 3. Aggregating by customer_ID to produce columns like: mean, std, min, max, last of that column
## 4. Joining labels corresponding to customer ID

In [ ]:
sampled_df = pd.read_csv("/content/drive/MyDrive/LSPDP Project/sampled_train_data_features_combined.csv")

In [ ]:
imp_cols = ['P_2','D_39','B_4','B_3','D_46',
            'B_5','R_1','D_42','S_3','B_2','B_1','P_3',
            'B_11','D_43','D_47','B_9','customer_ID']

In [ ]:
sampled_df_imp_cols = sampled_df.drop(columns=[col for col in sampled_df.columns if col not in imp_cols])

In [ ]:
sampled_df_imp_cols.shape

(553840, 17)

In [ ]:
sampled_df_imp_cols['customer_ID'] = sampled_df_imp_cols['customer_ID'].apply(lambda x: int(x[-16:], 16))

In [ ]:
grouped_sampled_df = sampled_df_imp_cols.groupby('customer_ID').agg(['mean', 'std', 'min', 'max', 'last'])

In [ ]:
grouped_sampled_df.columns = ['{}-{}'.format(col[0], col[1]) for col in grouped_sampled_df.columns]
grouped_sampled_df.reset_index(inplace=True)

In [ ]:
grouped_sampled_df.shape

(323851, 81)

In [ ]:
filtered_trained_labels = pd.read_csv("/content/drive/MyDrive/LSPDP Project/filtered_train_labels.csv")
filtered_trained_labels['customer_ID'] = filtered_trained_labels['customer_ID'].apply(lambda x: int(x[-16:], 16))

In [ ]:
filtered_trained_labels.shape

(323851, 2)

In [ ]:
merged_df = pd.merge(filtered_trained_labels,grouped_sampled_df, on='customer_ID')

In [ ]:
merged_df.shape

(323851, 82)

In [ ]:
merged_df.head(6)

,customer_ID,target,P_2-mean,P_2-std,P_2-min,P_2-max,P_2-last,D_39-mean,D_39-std,D_39-min,...,P_3-mean,P_3-std,P_3-min,P_3-max,P_3-last,B_11-mean,B_11-std,B_11-min,B_11-max,B_11-last
0,13914591055249847850,0,0.914767,NaN,0.914767,0.914767,0.914767,0.003029,NaN,0.003029,...,0.609996,NaN,0.609996,0.609996,0.609996,0.009845,NaN,0.009845,0.009845,0.009845
1,11750091188498716901,0,0.914856,0.006219,0.910458,0.919253,0.910458,0.122739,0.127922,0.032285,...,0.586891,0.033069,0.563508,0.610275,0.563508,0.010075,0.002869,0.008047,0.012104,0.008047
2,6537921148391624412,0,0.623392,NaN,0.623392,0.623392,0.623392,0.008380,NaN,0.008380,...,0.683057,NaN,0.683057,0.683057,0.683057,0.004926,NaN,0.004926,0.004926,0.004926
3,2065103583825424365,0,0.870445,0.012435,0.861652,0.879238,0.879238,0.005811,0.002352,0.004148,...,0.523304,0.066040,0.476607,0.570001,0.570001,0.008853,0.000695,0.008362,0.009344,0.009344
4,5835591494914154540,0,0.792005,0.010322,0.784706,0.799304,0.799304,0.285379,0.065336,0.239180,...,0.540277,0.060027,0.497831,0.582723,0.497831,0.012851,0.004775,0.009474,0.016227,0.009474
5,17981482738210298068,0,0.435276,0.036234,0.414269,0.477116,0.477116,0.003165,0.003074,0.000467,...,0.636282,0.111426,0.557492,0.715072,0.715072,0.035955,0.031122,0.003667,0.065763,0.003667


In [ ]:
merged_df.to_csv("/content/drive/MyDrive/LSPDP Project/sampled_dtree_training.csv", index=False)

In [5]:
df = pd.read_csv("/content/drive/MyDrive/LSPDP Project/sampled_dtree_training.csv")

In [15]:
df.shape

(323851, 82)

In [16]:
df.columns

Index(['customer_ID', 'target', 'P_2-mean', 'P_2-std', 'P_2-min', 'P_2-max',
       'P_2-last', 'D_39-mean', 'D_39-std', 'D_39-min', 'D_39-max',
       'D_39-last', 'B_1-mean', 'B_1-std', 'B_1-min', 'B_1-max', 'B_1-last',
       'B_2-mean', 'B_2-std', 'B_2-min', 'B_2-max', 'B_2-last', 'R_1-mean',
       'R_1-std', 'R_1-min', 'R_1-max', 'R_1-last', 'S_3-mean', 'S_3-std',
       'S_3-min', 'S_3-max', 'S_3-last', 'B_3-mean', 'B_3-std', 'B_3-min',
       'B_3-max', 'B_3-last', 'D_42-mean', 'D_42-std', 'D_42-min', 'D_42-max',
       'D_42-last', 'D_43-mean', 'D_43-std', 'D_43-min', 'D_43-max',
       'D_43-last', 'B_4-mean', 'B_4-std', 'B_4-min', 'B_4-max', 'B_4-last',
       'B_5-mean', 'B_5-std', 'B_5-min', 'B_5-max', 'B_5-last', 'D_46-mean',
       'D_46-std', 'D_46-min', 'D_46-max', 'D_46-last', 'D_47-mean',
       'D_47-std', 'D_47-min', 'D_47-max', 'D_47-last', 'B_9-mean', 'B_9-std',
       'B_9-min', 'B_9-max', 'B_9-last', 'P_3-mean', 'P_3-std', 'P_3-min',
       'P_3-max', 'P_3-last

In [17]:
df.drop('customer_ID', axis=1, inplace=True)

In [18]:
df.shape

(323851, 81)

In [19]:
df.columns

Index(['target', 'P_2-mean', 'P_2-std', 'P_2-min', 'P_2-max', 'P_2-last',
       'D_39-mean', 'D_39-std', 'D_39-min', 'D_39-max', 'D_39-last',
       'B_1-mean', 'B_1-std', 'B_1-min', 'B_1-max', 'B_1-last', 'B_2-mean',
       'B_2-std', 'B_2-min', 'B_2-max', 'B_2-last', 'R_1-mean', 'R_1-std',
       'R_1-min', 'R_1-max', 'R_1-last', 'S_3-mean', 'S_3-std', 'S_3-min',
       'S_3-max', 'S_3-last', 'B_3-mean', 'B_3-std', 'B_3-min', 'B_3-max',
       'B_3-last', 'D_42-mean', 'D_42-std', 'D_42-min', 'D_42-max',
       'D_42-last', 'D_43-mean', 'D_43-std', 'D_43-min', 'D_43-max',
       'D_43-last', 'B_4-mean', 'B_4-std', 'B_4-min', 'B_4-max', 'B_4-last',
       'B_5-mean', 'B_5-std', 'B_5-min', 'B_5-max', 'B_5-last', 'D_46-mean',
       'D_46-std', 'D_46-min', 'D_46-max', 'D_46-last', 'D_47-mean',
       'D_47-std', 'D_47-min', 'D_47-max', 'D_47-last', 'B_9-mean', 'B_9-std',
       'B_9-min', 'B_9-max', 'B_9-last', 'P_3-mean', 'P_3-std', 'P_3-min',
       'P_3-max', 'P_3-last', 'B_11-mean',

In [21]:
df.to_csv("/content/drive/MyDrive/LSPDP Project/sampled_dtree_training.csv", index=False)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323851 entries, 0 to 323850
Data columns (total 81 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   target     323851 non-null  int64  
 1   P_2-mean   321270 non-null  float64
 2   P_2-std    156946 non-null  float64
 3   P_2-min    321270 non-null  float64
 4   P_2-max    321270 non-null  float64
 5   P_2-last   321270 non-null  float64
 6   D_39-mean  323851 non-null  float64
 7   D_39-std   158379 non-null  float64
 8   D_39-min   323851 non-null  float64
 9   D_39-max   323851 non-null  float64
 10  D_39-last  323851 non-null  float64
 11  B_1-mean   323851 non-null  float64
 12  B_1-std    158379 non-null  float64
 13  B_1-min    323851 non-null  float64
 14  B_1-max    323851 non-null  float64
 15  B_1-last   323851 non-null  float64
 16  B_2-mean   323746 non-null  float64
 17  B_2-std    158299 non-null  float64
 18  B_2-min    323746 non-null  float64
 19  B_2-max    323746 non-n

In [24]:
df.isna().sum() != 0

target       False
P_2-mean      True
P_2-std       True
P_2-min       True
P_2-max       True
             ...  
B_11-mean    False
B_11-std      True
B_11-min     False
B_11-max     False
B_11-last    False
Length: 81, dtype: bool